In [3]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0
!pip install huggingface_hub
!pip install ipywidgets
!pip install -q gradio

Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.6 MB/s eta 0:00:00


In [4]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)
import ipywidgets as widgets
from IPython.display import display
import gradio as gr

In [5]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [6]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 1, max_steps = 100)
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 16, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

wiki_medical_terms_llam2.jsonl:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/6861 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
llama_sft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: biruktsegaye623 (biruktsegaye623-university-of-economics-in-katowice) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.7271926879882813, metrics={'train_runtime': 703.5868, 'train_samples_per_second': 0.142, 'train_steps_per_second': 0.142, 'total_flos': 2867999307890688.0, 'train_loss': 1.7271926879882813, 'epoch': 0.014575134819997084})

In [9]:
generator = pipeline("text-generation", model=llama_model, tokenizer=llama_tokenizer, max_length=500)

# In-memory user database
user_db = {}

# Response function
def generate_response(prompt):
    response = generator(f"<s>[INST] {prompt} [/INST]")[0]["generated_text"]
    return response

# Sign-up and login logic
def signup_user(new_username, new_password):
    if not new_username or not new_password:
        return "❌ No input. Please provide both username and password."
    if new_username in user_db:
        return "❌ Username already exists."
    user_db[new_username] = new_password
    return "✅ Account created! Please log in."

def login_user(username, password):
    if username in user_db and user_db[username] == password:
        return (
            gr.update(visible=True),   # Show chat UI
            gr.update(visible=False),  # Hide login UI
            "",                         # Clear login message
            gr.update(selected=3)      # Switch to Chat tab
        )
    return (
        gr.update(visible=False),
        gr.update(visible=True),
        "❌ Invalid credentials",
        gr.update(selected=2)
    )

def logout_user():
    return (
        gr.update(visible=False),  # Hide chat UI
        gr.update(visible=True),   # Show login UI
        gr.update(selected=0)      # Switch to Landing tab
    )

with gr.Blocks(theme="soft", css="""
#create-btn button,
#login-btn button,
#submit-btn button,
#logout-btn button {
    font-size: 12px !important;
    padding: 4px 8px !important;
    height: 30px !important;
    width: auto !important;
    min-width: 80px !important;
}
""") as demo:
    with gr.Tabs(selected=0, elem_id="tabs") as tabs:
        with gr.Tab("Home"):
            with gr.Column(elem_id="landing-container") as landing_ui:
                gr.Markdown("# 🏠 Welcome to MEDChat AI")
                gr.Markdown("---")
                gr.Markdown("#### 🔧 Features:")
                gr.Markdown("- Medical Q&A support\n- Easy-to-use chatbot interface\n- Privacy-focused with local data")
                gr.Markdown("---")
                gr.Markdown("© 2025 MEDChat AI | Finetuned by LLaMA 2 | Created for educational purposes")

        with gr.Tab("Sign Up"):
            with gr.Column() as signup_ui:
                gr.Markdown("### 📝 Sign Up")
                new_username = gr.Textbox(label="New Username")
                new_password = gr.Textbox(label="New Password", type="password")
                create_account_btn = gr.Button("Create Account", elem_id="create-btn")
                signup_msg = gr.Markdown()

        with gr.Tab("Login"):
            with gr.Column(visible=True) as login_ui:
                gr.Markdown("### 🔐 Login")
                username = gr.Textbox(label="Username")
                password = gr.Textbox(label="Password", type="password")
                login_btn = gr.Button("Login", elem_id="login-btn")
                login_msg = gr.Markdown()

        with gr.Tab("Chat"):
            with gr.Column(visible=False) as chat_ui:
                gr.Markdown("## 💬 MEDChat AI")
                gr.Markdown("What can I help with today?")
                logout_btn = gr.Button("🚪 Logout", elem_id="logout-btn")
                prompt = gr.Textbox(lines=5, placeholder="Enter your prompt...")
                submit_btn = gr.Button("Submit", elem_id="submit-btn")
                response = gr.Textbox(label="Response")

                gr.Markdown("### Try one of these:")
                examples = gr.Examples(
                    examples=[
                        ["What does the immune system do?"],
                        ["What is Epistaxis?"],
                        ["Do our intestines contain germs?"],
                        ["What are allergies?"],
                        ["Should I start taking creatine?"],
                        ["What are antibiotics?"],
                        ["Why do I get sick?"],
                        ["What's the difference between bacteria and viruses?"],
                        ["Where are some places that germs hide?"],
                    ],
                    inputs=prompt
                )



                logout_btn.click(fn=logout_user, outputs=[chat_ui, login_ui, tabs])

    # Tab navigation logic
    to_signup = gr.Button(visible=False)
    to_login = gr.Button(visible=False)

    to_signup.click(lambda: gr.update(selected=1), outputs=tabs)
    to_login.click(lambda: gr.update(selected=2), outputs=tabs)
    create_account_btn.click(fn=signup_user, inputs=[new_username, new_password], outputs=signup_msg)
    login_btn.click(
        fn=login_user,
        inputs=[username, password],
        outputs=[chat_ui, login_ui, login_msg, tabs]
    )
    submit_btn.click(fn=generate_response, inputs=prompt, outputs=response)

# Launch the interface
demo.launch(share=True)

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74b43cb1296c900af1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
